In [1]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import classification_report, mean_squared_error
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

In [2]:

base_path = '/kaggle/input/facial-age/face_age/'
image_folders = os.listdir(base_path)

# print(image_folders)

age_labels = []
gender_labels = []
image_paths = []
image_names = []

for dir_name in image_folders:
    dir_path = os.path.join(base_path, dir_name)
    image_files = os.listdir(dir_path)
    for img in image_files:
        try:
            age = int(dir_name)
            age_labels.append(age)
            image_names.append(img)
            image_paths.append(os.path.join(base_path,dir_name,img))
        except Exception as e:
            # the dataset folder has another folder with the dataset itself: This will prevent that folder from joining the df
            continue
            

df = pd.DataFrame({
    'image_path': image_paths,
    'image_name': image_names,
    'age': age_labels
})

df

,image_path,image_name,age
0,/kaggle/input/facial-age/face_age/057/6802.png,6802.png,57
1,/kaggle/input/facial-age/face_age/057/3702.png,3702.png,57
2,/kaggle/input/facial-age/face_age/057/8810.png,8810.png,57
3,/kaggle/input/facial-age/face_age/057/6759.png,6759.png,57
4,/kaggle/input/facial-age/face_age/057/1846.png,1846.png,57
...,...,...,...
9773,/kaggle/input/facial-age/face_age/085/7733.png,7733.png,85
9774,/kaggle/input/facial-age/face_age/085/3123.png,3123.png,85
9775,/kaggle/input/facial-age/face_age/085/5302.png,5302.png,85
9776,/kaggle/input/facial-age/face_age/085/9632.png,9632.png,85


In [3]:
df['Age_Group'] = pd.qcut(df['age'], q=4, labels=range(0,4))
df['Age_Interval'] = pd.qcut(df['age'], q=4)

df

,image_path,image_name,age,Age_Group,Age_Interval
0,/kaggle/input/facial-age/face_age/057/6802.png,6802.png,57,3,"(49.0, 110.0]"
1,/kaggle/input/facial-age/face_age/057/3702.png,3702.png,57,3,"(49.0, 110.0]"
2,/kaggle/input/facial-age/face_age/057/8810.png,8810.png,57,3,"(49.0, 110.0]"
3,/kaggle/input/facial-age/face_age/057/6759.png,6759.png,57,3,"(49.0, 110.0]"
4,/kaggle/input/facial-age/face_age/057/1846.png,1846.png,57,3,"(49.0, 110.0]"
...,...,...,...,...,...
9773,/kaggle/input/facial-age/face_age/085/7733.png,7733.png,85,3,"(49.0, 110.0]"
9774,/kaggle/input/facial-age/face_age/085/3123.png,3123.png,85,3,"(49.0, 110.0]"
9775,/kaggle/input/facial-age/face_age/085/5302.png,5302.png,85,3,"(49.0, 110.0]"
9776,/kaggle/input/facial-age/face_age/085/9632.png,9632.png,85,3,"(49.0, 110.0]"


In [4]:
import torchvision.transforms as transforms
import torch
import torch.nn as nn
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import pandas as pd
import os


transform_pipeline = transforms.Compose([
    transforms.ToTensor(),                     # Convert image to tensor
    # transforms.ToPILImage(),                   # Convert OpenCV image (NumPy array) to PIL format
    transforms.RandomHorizontalFlip(p=0.5),    # 50% chance of horizontal flip
    transforms.RandomRotation(30),             # Random rotation within ±30 degrees
    transforms.Resize((224, 224)),             # Resize to desired input size (224x224 here)
    # transforms.Normalize([0.5], [0.5])         # Normalize; mean and std of 0.5 for grayscale
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    transforms.ColorJitter(brightness=0.1, contrast=0.1)
])

In [5]:
def test_processing(df):
    brightImages = []
    smallImages = []
    dullImages = []
    
    blist = []
    varList = []
    for idx in range(len(df)):
        img_path = df.iloc[idx]['image_path']
        img_path = os.path.join(img_path)
        label = df.iloc[idx]['Age_Group']  # Age group label for classification

        image = Image.open(img_path)
        img_array = np.array(image)

        brightness = np.mean(img_array)
        blist.append(brightness)
        if brightness < 50 or brightness > 225:  
            brightImages.append(idx)

        if img_array.shape[0] < 50 or img_array.shape[1] < 50:  # Example minimum size
            smallImages.append(idx)
            
        gray = cv2.cvtColor(img_array, cv2.COLOR_BGR2GRAY)
        variance = cv2.Laplacian(gray, cv2.CV_64F).var()
        varList.append(variance)
        if variance < 22.814826998125:
            dullImages.append(idx)
            
    
    # print(brightImages)
    # print(smallImages)
    # print(dullImages)
    return blist, varList, dullImages
            
            
blist, varlist, dullImages = test_processing(df)



In [6]:

model = models.vgg16(pretrained=None)


num_classes = 5

# model.classifier[6] = nn.Linear(4096, num_classes)
num_ftrs = model.classifier[-1].in_features
model.classifier[-1] = nn.Linear(num_ftrs, num_classes)
feature_extractor = torch.nn.Sequential(*(list(model.children())[:-1]))

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = model.to(device)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [7]:

class facialAge(Dataset):
    def __init__(self, df, transform=None, preprocess=None):
        self.df = df
        self.transform = transform
        self.preprocess = preprocess

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = self.df.iloc[idx]['image_path']
        img_path = os.path.join(img_path)
        label = self.df.iloc[idx]['Age_Group']  # Age group label for classification

        image = Image.open(img_path)
        if self.preprocess:
            image = self.preprocess(image)
        if self.transform:
            image = self.transform(image)

        return image, label

    
    
train_df, test_df = train_test_split(df, test_size=0.2, random_state=21)

train_dataset = facialAge(train_df, transform_pipeline)
test_dataset = facialAge(test_df, transform_pipeline)

train_df

,image_path,image_name,age,Age_Group,Age_Interval
6508,/kaggle/input/facial-age/face_age/011/3280.png,3280.png,11,1,"(7.0, 25.0]"
5595,/kaggle/input/facial-age/face_age/066/4167.png,4167.png,66,3,"(49.0, 110.0]"
6565,/kaggle/input/facial-age/face_age/004/8794.png,8794.png,4,0,"(0.999, 7.0]"
279,/kaggle/input/facial-age/face_age/053/8225.png,8225.png,53,3,"(49.0, 110.0]"
4218,/kaggle/input/facial-age/face_age/034/6673.png,6673.png,34,2,"(25.0, 49.0]"
...,...,...,...,...,...
9336,/kaggle/input/facial-age/face_age/038/5783.png,5783.png,38,2,"(25.0, 49.0]"
48,/kaggle/input/facial-age/face_age/057/2579.png,2579.png,57,3,"(49.0, 110.0]"
8964,/kaggle/input/facial-age/face_age/010/6241.png,6241.png,10,1,"(7.0, 25.0]"
5944,/kaggle/input/facial-age/face_age/002/955.png,955.png,2,0,"(0.999, 7.0]"


Lr 0.001 Batch size 128

In [8]:
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support


dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=128, shuffle=True)

import torch
import torch.optim as optim

learning_rates = [0.001, 0.0001, 0.00001, 0.000001, 0.0000001]
# Set up loss function and optimizer
test_accuracies_across_learning_rates = []
for lr in learning_rates:

    model = models.vgg16(pretrained=True)


    num_classes = 5

    # model.classifier[6] = nn.Linear(4096, num_classes)
    num_ftrs = model.classifier[-1].in_features
    model.classifier[-1] = nn.Linear(num_ftrs, num_classes)
    feature_extractor = torch.nn.Sequential(*(list(model.children())[:-1]))

    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    model = model.to(device)

    print("Learning Rate: " + str(lr))
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    num_epochs = 10
    device ='cuda'

    for epoch in range(num_epochs):
        total_train = 0
        correct_train = 0
        model.train()
        running_loss = 0.0

        for batch in dataloader:
            images, labels = batch  # Unpack images and labels
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            _, predicted = torch.max(outputs, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

            running_loss += loss.item()
            correct_train += (predicted == labels).sum().item()
            total_train += labels.size(0)

        train_accuracy = 100 * correct_train / total_train

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(dataloader):.4f}, Train Accuracy: {train_accuracy}")


    model.eval()

    # Initialize variables to track performance
    correct = 0
    total = 0
    test_loss = 0.0

    all_labels = []
    all_predictions = []

    # Disable gradient computation for testing
    with torch.no_grad():
        for batch in test_dataloader:  # Assuming test_dataloader is your DataLoader for the test set
            images, labels = batch
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item()

            # Get predicted class by taking the class with the highest score
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            # Append to lists (convert to CPU and detach from computation graph)
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

    # Calculate average loss and accuracy
    average_test_loss = test_loss / len(test_dataloader)
    accuracy = 100 * correct / total
    test_accuracies_across_learning_rates.append(accuracy)
    print(f'Test Loss: {average_test_loss:.4f}, Accuracy: {accuracy:.2f}%')





    conf_matrix = confusion_matrix(all_labels, all_predictions)



    precision, recall, f1_score, support = precision_recall_fscore_support(all_labels, all_predictions)

    print("\nPrecision per class:", precision)

    print("Recall per class:", recall)

    print("F1-score per class:", f1_score)

    print("Samples per class:", support)

    print("----------------------------------------------------------------------")


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:02<00:00, 197MB/s] 


Learning Rate: 0.001
Epoch [1/10], Loss: 1.5841, Train Accuracy: 26.208130912810024
Epoch [2/10], Loss: 1.2933, Train Accuracy: 37.138839171567376
Epoch [3/10], Loss: 1.0801, Train Accuracy: 51.24009204806955
Epoch [4/10], Loss: 0.9747, Train Accuracy: 57.47890565072871
Epoch [5/10], Loss: 0.9229, Train Accuracy: 58.25875735106111
Epoch [6/10], Loss: 0.8762, Train Accuracy: 61.761697775504985
Epoch [7/10], Loss: 0.8695, Train Accuracy: 61.902326770646894
Epoch [8/10], Loss: 0.8055, Train Accuracy: 64.44643313730504
Epoch [9/10], Loss: 0.7900, Train Accuracy: 65.18793147532601
Epoch [10/10], Loss: 0.7593, Train Accuracy: 66.83712605471746
Test Loss: 0.7179, Accuracy: 69.07%

Precision per class: [0.86148008 0.59356137 0.58823529 0.68458781]
Recall per class: [0.88326848 0.61076605 0.45548654 0.80252101]
F1-score per class: [0.87223823 0.60204082 0.5134189  0.73887814]
Samples per class: [514 483 483 476]
----------------------------------------------------------------------


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Learning Rate: 0.0001
Epoch [1/10], Loss: 0.9096, Train Accuracy: 60.2019943748402
Epoch [2/10], Loss: 0.6098, Train Accuracy: 73.9580669905395
Epoch [3/10], Loss: 0.5400, Train Accuracy: 77.24367169521861
Epoch [4/10], Loss: 0.4920, Train Accuracy: 79.16133981079008
Epoch [5/10], Loss: 0.4764, Train Accuracy: 79.9667604193301
Epoch [6/10], Loss: 0.4149, Train Accuracy: 82.65149578113015
Epoch [7/10], Loss: 0.4039, Train Accuracy: 83.30350294042444
Epoch [8/10], Loss: 0.3506, Train Accuracy: 85.61748913321401
Epoch [9/10], Loss: 0.3192, Train Accuracy: 86.84479672717976
Epoch [10/10], Loss: 0.2861, Train Accuracy: 88.0848887752493
Test Loss: 0.5160, Accuracy: 79.24%

Precision per class: [0.97840173 0.74007937 0.62950257 0.87931034]
Recall per class: [0.88132296 0.77225673 0.75983437 0.75      ]
F1-score per class: [0.92732856 0.75582573 0.68855535 0.80952381]
Samples per class: [514 483 483 476]
----------------------------------------------------------------------


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Learning Rate: 1e-05
Epoch [1/10], Loss: 1.2040, Train Accuracy: 45.525441063666584
Epoch [2/10], Loss: 0.8445, Train Accuracy: 63.51316798772692
Epoch [3/10], Loss: 0.7327, Train Accuracy: 69.31731015085656
Epoch [4/10], Loss: 0.6761, Train Accuracy: 71.50345180260803
Epoch [5/10], Loss: 0.6423, Train Accuracy: 73.11429301968806
Epoch [6/10], Loss: 0.6011, Train Accuracy: 74.80184096139095
Epoch [7/10], Loss: 0.5886, Train Accuracy: 75.10866785988239
Epoch [8/10], Loss: 0.5603, Train Accuracy: 76.52774226540527
Epoch [9/10], Loss: 0.5261, Train Accuracy: 77.8828944004091
Epoch [10/10], Loss: 0.5100, Train Accuracy: 78.58603937611863
Test Loss: 0.5415, Accuracy: 76.89%

Precision per class: [0.89980732 0.68809074 0.63829787 0.87723785]
Recall per class: [0.90856031 0.75362319 0.68322981 0.72058824]
F1-score per class: [0.90416263 0.71936759 0.66       0.79123414]
Samples per class: [514 483 483 476]
----------------------------------------------------------------------


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Learning Rate: 1e-06
Epoch [1/10], Loss: 1.5857, Train Accuracy: 25.632830478138583
Epoch [2/10], Loss: 1.4276, Train Accuracy: 34.760930708258755
Epoch [3/10], Loss: 1.3132, Train Accuracy: 41.3577090258246
Epoch [4/10], Loss: 1.2068, Train Accuracy: 47.32804909230376
Epoch [5/10], Loss: 1.1123, Train Accuracy: 52.17335719764766
Epoch [6/10], Loss: 1.0541, Train Accuracy: 54.15494758373818
Epoch [7/10], Loss: 0.9929, Train Accuracy: 57.018665302991565
Epoch [8/10], Loss: 0.9638, Train Accuracy: 58.987471234978265
Epoch [9/10], Loss: 0.9210, Train Accuracy: 60.3426233699821
Epoch [10/10], Loss: 0.8982, Train Accuracy: 61.00741498338021
Test Loss: 0.8160, Accuracy: 64.88%

Precision per class: [0.77333333 0.57112527 0.51287129 0.73626374]
Recall per class: [0.78988327 0.55693582 0.53623188 0.70378151]
F1-score per class: [0.78152069 0.5639413  0.5242915  0.71965628]
Samples per class: [514 483 483 476]
----------------------------------------------------------------------


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Learning Rate: 1e-07
Epoch [1/10], Loss: 1.6462, Train Accuracy: 22.296087957044236
Epoch [2/10], Loss: 1.6170, Train Accuracy: 23.843006903605215
Epoch [3/10], Loss: 1.5936, Train Accuracy: 24.622858603937612
Epoch [4/10], Loss: 1.5739, Train Accuracy: 25.28765021733572
Epoch [5/10], Loss: 1.5560, Train Accuracy: 27.831756583993865
Epoch [6/10], Loss: 1.5334, Train Accuracy: 27.767834313474815
Epoch [7/10], Loss: 1.5153, Train Accuracy: 29.059064177959602
Epoch [8/10], Loss: 1.4961, Train Accuracy: 30.28637177192534
Epoch [9/10], Loss: 1.4845, Train Accuracy: 31.33469700843774
Epoch [10/10], Loss: 1.4694, Train Accuracy: 31.654308361032985
Test Loss: 1.4196, Accuracy: 39.16%

Precision per class: [0.54054054 0.32817337 0.29806259 0.41580042]
Recall per class: [0.50583658 0.2194617  0.41407867 0.42016807]
F1-score per class: [0.52261307 0.2630273  0.34662045 0.41797283]
Samples per class: [514 483 483 476]
----------------------------------------------------------------------
